# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

The Chain is the most important part of LangChain, it combines an LLM with the prompt, and with this block you creat a bunch of them to carry an operation in your text

In [12]:
import os
import os
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
API_BASE = os.getenv('AZURE_OPENAI_ENDPOINT')
MODEL = os.getenv('OPENAI_MODEL_NAME')
API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')

import warnings
warnings.filterwarnings('ignore')

##########################
from langchain_openai import AzureChatOpenAI

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [ ]:
#!pip install pandas

In [6]:
text = '''España es un país situado en el suroeste de Europa, 
en la península ibérica. Limita al norte con Francia y Andorra, 
al oeste con Portugal, y al sur con el mar Mediterráneo y el océano Atlántico.
 Su capital es Madrid, una ciudad vibrante y llena de historia.

España es conocida por su rica cultura y patrimonio. 
Desde la arquitectura de Gaudí en Barcelona hasta los museos de arte en Madrid, 
como el Museo del Prado, el país ofrece una gran variedad de experiencias culturales. 
Además, España es famosa por su gastronomía, que incluye platos como la paella, 
el gazpacho y las tapas.'''

## LLMChain

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
#First we initialize the LLM with a high temperature to get a fun descriptions

#This doesnt work -> llm = ChatOpenAI(temperature=0.9, model=MODEL)

llm = AzureChatOpenAI(temperature=0.9, 
                      api_key=API_KEY, 
                      api_version=API_VERSION,
                      azure_endpoint=API_BASE)

In [17]:
#Then we initialazie the prompt, is gonna take a varible product...
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?\
        JUST RETURN THE NAME UP TO 6 WORDS"
)

In [18]:
#Now we are gonna use a chain to combine these two things.
#We create the chain object.

chain = LLMChain(llm=llm, prompt=prompt)

In [19]:
product = "Queen Size Sheet Set"

# Then we run the Chain object we´ve created before
chain.run(product)

'Royal Rest Linens'

## SimpleSequentialChain

Sequential chains, run a sequence of chains one after another.

The output of one chain is the input of another chain

There is two type of sequential chains:
* Simple sequetial chains : Single input/outpu
* SequentialChains: Multiuple inputs/outputs

In [20]:
from langchain.chains import SimpleSequentialChain

In [21]:
#Create LLM object
#This doesnt work -->>> llm = ChatOpenAI(temperature=0.9, model=llm_model)
llm = AzureChatOpenAI(temperature=0.9, 
                      api_key=API_KEY, 
                      api_version=API_VERSION,
                      azure_endpoint=API_BASE)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [22]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [23]:
#We create the sequential chain by joining chain_one and chain_two
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)

In [24]:
#Now we execute it for getting some reuslts
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Choosing the best name for a company that specializes in Queen Size Sheet Sets involves considering factors like brand identity, target audience, and uniqueness. Here are a few suggestions that might resonate well:

1. **Queen's Comfort**
2. **Royal Rest Linens**
3. **Majestic Sheets**
4. **Queen's Haven Bedding**
5. **Regal Slumber Sheets**
6. **Crown Sleep Sets**
7. **Monarch Linens**
8. **Queen's Dream Sheets**
9. **Sovereign Sleep Co.**
10. **Imperial Bedding Solutions**

Remember to check for trademark availability and domain registration to ensure the name can be uniquely yours.
A company specializing in Queen Size Sheet Sets, focused on luxury, comfort, and elegance, offering uniquely branded bedding solutions.

> Finished chain.


'A company specializing in Queen Size Sheet Sets, focused on luxury, comfort, and elegance, offering uniquely branded bedding solutions.'

## SequentialChain

Multiple inputs/outputs

In [25]:
from langchain.chains import SequentialChain

In [26]:
#This doesnt work -->>> llm = ChatOpenAI(temperature=0.9, model=llm_model)
llm = AzureChatOpenAI(temperature=0.9, 
                      api_key=API_KEY, 
                      api_version=API_VERSION,
                      azure_endpoint=API_BASE)

#For first chane, we´re gona take the review and translate it to english

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


Very inportant that the input keys and output keys must be presicely name correctly and similar.

1st Chain output_key -> Englis_Review //  2nd Chain input_key -> Englis_Review

In [27]:
#Second chain create a summary of that review tranlation

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [28]:
#3rd chain will return the original language of that review

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [29]:
#4th chain will take multiple inputs, summary input, and original language.and
# And it will return a summary in the original language
 
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [30]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [31]:

overall_chain(text)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'España es un país situado en el suroeste de Europa, \nen la península ibérica. Limita al norte con Francia y Andorra, \nal oeste con Portugal, y al sur con el mar Mediterráneo y el océano Atlántico.\n Su capital es Madrid, una ciudad vibrante y llena de historia.\n\nEspaña es conocida por su rica cultura y patrimonio. \nDesde la arquitectura de Gaudí en Barcelona hasta los museos de arte en Madrid, \ncomo el Museo del Prado, el país ofrece una gran variedad de experiencias culturales. \nAdemás, España es famosa por su gastronomía, que incluye platos como la paella, \nel gazpacho y las tapas.',
 'English_Review': "Spain is a country located in the southwest of Europe, on the Iberian Peninsula. It borders France and Andorra to the north, Portugal to the west, and the Mediterranean Sea and the Atlantic Ocean to the south. Its capital is Madrid, a vibrant city full of history.\n\nSpain is known for its rich culture and heritage. From Gaudí's architecture in Barcelona to the art

## Router Chain

For more complicate operation, 
when you have multiple subchains and each one is especializied in a particular input.

You can have a router Chain , which decides wich subchain to use

the router_chain typically uses a language model (LLM) to analyze the input and determine which destination_chain to route the request to. The LLM evaluates the input based on predefined criteria or prompts and then selects the most appropriate subchain to handle the task.

In [ ]:
#Here we have some subjects templates, the main idea is: When the Router chain calls
# a subchai , this chain should be the one related with the field 


physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
# This will indicate the Router chain which template promt it should it use base 
# on the follow description.


prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [34]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [35]:
#This doesnt work -->>> llm = ChatOpenAI(temperature=0.9, model=llm_model)
llm = AzureChatOpenAI(temperature=0.9, 
                      api_key=API_KEY, 
                      api_version=API_VERSION,
                      azure_endpoint=API_BASE)

In [ ]:
#These are the chains that will be called by the router Chain,
# here we are creatin all the subchains

destination_chains = {}


# prompt_infos is expected to be a list of dictionaries, 
# where each dictionary contains information about a specific prompt, 
# including its name and template.

for p_info in prompt_infos:


    #Extract information and create chains

    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  

    #All sub chains have been added to the dict 'destination_chains'
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [64]:
destinations

['physics: Good for answering questions about physics',
 'math: Good for answering math questions',
 'History: Good for answering history questions',
 'computer science: Good for answering computer science questions']

In [62]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [60]:
#This is the Default Chain, when the Router Chain doesnt know wich chain to call,
# by default it will call this chain

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [61]:
#This is the template that is gonna be used by the LLM


MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

The **format** method is used to replace the placeholder {destinations} in the MULTI_PROMPT_ROUTER_TEMPLATE with the value of destinations_str.

In [46]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

This code snippet is creating a PromptTemplate for the router, which will use the router_template to decide how to route inputs to the appropriate destination_chain.

* template=router_template: This uses the router_template created earlier, which includes the formatted list of destination chains.

* input_variables=["input"]: This specifies that the template will take an input variable. This is the input that the router will analyze to decide which subchain to route to.

* output_parser=RouterOutputParser(): This is crucial. The RouterOutputParser is responsible for interpreting the output of the language model and determining which destination_chain to use based on the input.

In [ ]:

#Here we create the router Prompt, by passing it the 'router_template' as argument
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],

    #IMPORTAT WE HAVE HERE THE RouterOutputParser
    #It will help the router to decide wich subchanges to route
    output_parser=RouterOutputParser(),
)


#Here we create our Router Chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
# Finally we can create the overall chain
# it has the router chain, destination chain and default chain

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )

In [58]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by an idealized object known as a black body, which absorbs all incident radiation, regardless of frequency or angle of incidence. A perfect black body is a theoretical construct that emits radiation in a characteristic spectrum solely determined by its temperature, according to Planck's law. \n\nThis radiation has a continuous spectrum, starting from near zero at low frequencies, rising to a peak at a frequency that increases with temperature, and then decreasing again at higher frequencies. The radiation includes all wavelengths, and the peak of the emission shifts to shorter wavelengths as the temperature of the black body increases—this is known as Wien’s Displacement Law.\n\nAn excellent real-world approximation of a black body is a cavity with a small hole; any radiation entering the hole is unlikely to escape, as it will be absorbed or reflected multiple times within the cavity. The concept of black body radia

In [44]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2?'}
> Finished chain.


'To solve the question "What is 2 + 2?", let\'s break it down:\n\n1. Identify the numbers involved: We have two instances of the number 2.\n2. Understand the operation: The operation here is addition, which involves combining the values of the numbers.\n3. Perform the addition: Add the first 2 to the second 2.\n\n\\[ 2 + 2 = 4 \\]\n\nSo, the answer is 4.'

In [45]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA carries the genetic instructions necessary for the development, functioning, growth, and reproduction of all living organisms, including humans. Here are some key reasons why DNA is present in nearly every cell:\n\n1. **Genetic Blueprint:** DNA contains the genetic blueprint or instructions needed for the synthesis of proteins, which are crucial for the structure and function of cells. Proteins perform most of the work within cells and are essential for the body's structure, function, and regulation of tissues and organs.\n\n2. **Cell Function and Identity:** DNA provides the information needed for cells to carry out their specific roles. Different cell types activate different genes within the DNA, allowing them to perform specialized functions, such as muscle contraction or nerve impulse transmission.\n\n3. **Reproduction and Growth:** DNA is necessary for cell division and growth. When cells divide, DNA is replicated so that each new 